In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import os
from torchvision import models


### Sanity Check for GPU Runtime

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # use gpu if available
# print device name if using gpu with message "Using GPU: device_name"
if device.type == "cuda":
    print("Using GPU: {}".format(torch.cuda.get_device_name(0)))


Using GPU: NVIDIA GeForce GTX 1070 Ti


### Transform and Split the Dataset


In [4]:
# Data augmentation
transform_train = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # normalize in range [-1,1]
])

# Define a function to load and preprocess images
def load_and_preprocess_image(file_path):
    # Open the image file
    with Image.open(file_path) as img:
        # Apply the transformations
        img_transformed = transform_train(img)  # use transform_train instead of transform
    return img_transformed

# Define the directories where your images are stored
covid_dir = "./dataset_14/covid"
normal_dir = "./dataset_14/normal"

# Load and preprocess all images
data = []
labels = []
for filename in os.listdir(covid_dir):
    img_tensor = load_and_preprocess_image(os.path.join(covid_dir, filename))
    data.append(img_tensor)
    labels.append(1)  # '1' for 'covid'
for filename in os.listdir(normal_dir):
    img_tensor = load_and_preprocess_image(os.path.join(normal_dir, filename))
    data.append(img_tensor)
    labels.append(0)  # '0' for 'normal'

# Convert the data and labels to PyTorch tensors
data = torch.stack(data)
labels = torch.tensor(labels)

# First, split the data into a training set and a temporary set using an 80-20 split
train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Then, split the temporary set into a validation set and a test set using a 50-50 split
val_data, test_data, val_labels, test_labels = train_test_split(temp_data, temp_labels, test_size=0.5, random_state=42)

In [5]:
train_dataset = TensorDataset(train_data, train_labels)
val_dataset = TensorDataset(val_data, val_labels)
test_dataset = TensorDataset(test_data, test_labels)

### Create DataLoaders

In [6]:
# Create DataLoaders
batch_size = 32 # Parameter: batch size
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)




### CNN


In [6]:
# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            # conv1
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),  # Changed in_channels to 1
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            # conv2
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            # conv3
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            # conv4
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
        )

        self.dense_layers = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(50176, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 2),  # Changed K to 2
        )

    def forward(self, X):
        out = self.conv_layers(X)

    # Flatten the output of the conv layers
        out = out.view(out.size(0), -1)
    
    # Fully connected
        out = self.dense_layers(out)
        return out



    
net = CNN()
net = net.to(device) # move the network to GPU if available

## Transfer Learning with VGG-16

In [8]:
# Load pre-trained VGG16 model
net = models.vgg16(pretrained=True)

# Modify the last layer
num_features = net.classifier[6].in_features
net.classifier[6] = nn.Linear(num_features, 2)  # '2' for 'covid' and 'normal'

# Move model to GPU if available
net = net.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# initialize tensorboard
writer = SummaryWriter('runs/covid_classifier_experiment_1')

# Train the model
for epoch in range(30):  # loop over the dataset multiple times
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 20 mini-batches
            print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

    # log the epoch loss and accuracy
    writer.add_scalar('Loss/train', running_loss, epoch)
    writer.add_scalar('Accuracy/train', 100 * correct / total, epoch)

# Accuracy test and image comparison on validation set
correct = 0
total = 0
with torch.no_grad():
        for data in val_dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

            # Add image comparisons to TensorBoard
        img_grid = torchvision.utils.make_grid(images)
        writer.add_image('covid_vs_normal', img_grid)

print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the validation images: 81 %


### Training Loop


Finished Training


### Test the network on the test data

In [26]:

# Accuracy test and image comparison on validation set
correct = 0
total = 0
with torch.no_grad():
        for data in val_dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Add image comparisons to TensorBoard
        img_grid = torchvision.utils.make_grid(images)
        writer.add_image('covid_vs_normal', img_grid)

        writer.add_scalar('Accuracy/validation', 100 * correct / total, epoch)
        writer.add_scalar('Loss/validation', running_loss, epoch)

print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the validation images: 72 %
